In [165]:
# !pip3 install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL for the SeattleSeq SNV annotation tool
url = "https://cadd.gs.washington.edu/snv"
df = pd.read_csv('O00555-variants.csv')
results_df = df.copy()
results_df['RawScore'] = None
results_df['PHRED'] = None

df.head()

,Chromosome,NTPosition,NTChange,AAPosition,AAChange,Phenotype
0,19,13506221,C>A,2,A2S,"Developmental and epileptic encephalopathy, 42..."
1,19,13506220,G>A,2,A2V,"Developmental and epileptic encephalopathy, 42..."
2,19,13506218,G>A,3,R3C,"Developmental and epileptic encephalopathy, 42..."
3,19,13506218,G>C,3,R3G,"Developmental and epileptic encephalopathy, 42..."
4,19,13506211,C>A,5,G5V,NaN


In [166]:
for index in range(df.shape[0]):
# Example payload for a query (this may need to be adjusted based on actual site input fields)
    payload = {
        'chr': df.loc[index].Chromosome,
        'pos': df.loc[index].NTPosition,
        'ref': df.loc[index].NTChange[0],
        'alt': df.loc[index].NTChange[2],
        'version': 'GRCh38-v1.7'
    }

    # Send the query to the website
    response = requests.post(url, data=payload)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find and extract the relevant information
        results = soup.find(id='snv-data').text.split('\n')[1:-1]  # Adjust this based on actual site structure
        scores = results[-2:]
        r_s = float(scores[0][8:])
        phred = float(scores[1][5:])
        results_df.loc[index, "RawScore"] = r_s
        results_df.loc[index, "PHRED"] = phred
        print('Index ', index,' Complete')
    else:
        print(f"Error: {response.status_code}")
    
    time.sleep(10)

Index  0  Complete


KeyboardInterrupt: 

In [167]:
results_df.head()

,Chromosome,NTPosition,NTChange,AAPosition,AAChange,Phenotype,RawScore,PHRED
0,19,13506221,C>A,2,A2S,"Developmental and epileptic encephalopathy, 42...",3.606917,23.0
1,19,13506220,G>A,2,A2V,"Developmental and epileptic encephalopathy, 42...",None,None
2,19,13506218,G>A,3,R3C,"Developmental and epileptic encephalopathy, 42...",None,None
3,19,13506218,G>C,3,R3G,"Developmental and epileptic encephalopathy, 42...",None,None
4,19,13506211,C>A,5,G5V,NaN,None,None


In [148]:
results_df.to_csv("Variant Results.csv")